<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h2><center>Семинар: линейные модели</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Линейная регрессия

## Пример: Стоимость автомобиля

Загрузите тренировочные данные и тестовые данные - уже знакомые нам данные по автомобилям.

In [ ]:
df_train = pd.read_csv('./data/accord_sedan_training.csv')
df_test = pd.read_csv('./data/accord_sedan_testing.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.plot(x='mileage', y='price', kind='scatter', s=120)

Кажется, что между стоимостью и пробегом зависимость линейная - давайте ее найдем!

In [ ]:
X_train = df_train.mileage.values.reshape(-1, 1)
y_train = df_train.price.values

In [ ]:
from sklearn.linear_model import LinearRegression

Обучим модель

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
print('Модель:\nprice = %.2f + (%.2f)*mileage' % (model.intercept_, model.coef_[0]))

Нарисуйте предсказание модели (прямую) вместе с данными на плоскости. Здесь можно либо явно взять уравнение прямой и посчитать значения в каждой точке, либо через predict.

In [ ]:
df_train.plot(x='mileage', y='price', kind='scatter', s=120)

## Your Code Here

## Меры качества


**1. (R)MSE ((Root) Mean Squared Error)**

$$ L(\hat{y}, y) = \frac{1}{N}\sum\limits_n^N (y_n - \hat{y}_n)^2$$

**2. MAE (Mean Absolute Error)**

$$ L(\hat{y}, y) = \frac{1}{N}\sum\limits_n^N |y_n - \hat{y}_n|$$

* Чем плохи такие метрики?
* Отличия - http://yahwes.github.io/2016/03/22/mae-rmse/

**3. RSE (Relative Squared Error)**

$$ L(\hat{y}, y) = \sqrt\frac{\sum\limits_n^N (y_n - \hat{y}_n)^2}{\sum\limits_n^N (y_n - \bar{y})^2}$$

**4. RAE (Relative Absolute Error)**

$$ L(\hat{y}, y) = \frac{\sum\limits_n^N |y_n - \hat{y}_n|}{\sum\limits_n^N |y_n - \bar{y}|}$$

**5. MAPE (Mean Absolute Persentage Error)**

$$ L(\hat{y}, y) = \frac{100}{N} \sum\limits_n^N\left|\frac{ y_n - \hat{y}_n}{y_n}\right|$$




**6. RMSLE (Root Mean Squared Logarithmic Error)**

$$ L(\hat{y}, y) = \sqrt{\frac{1}{N}\sum\limits_n^N(\log(y_n + 1) - \log(\hat{y}_n + 1))^2}$$

In [ ]:
y = 10000
y_hat = np.linspace(0, 30000, 151)
# log error
error1 = np.sqrt((np.log(y+1) - np.log(y_hat + 1))**2)

# squared error
error2 = (y - y_hat)**2 /1000.

plt.plot(y_hat, error1, label='RMSLE')
plt.plot(y_hat, error2, label='MSE')
plt.xlabel('$\hat{y}$')
plt.ylabel('Error')
plt.title('true value y = %.1f' % y)
plt.legend()
plt.ylim(0, 10)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
y_hat = model.predict(X_train)

In [ ]:
print('Средняя абсолютная ошибка %.2f' % mean_absolute_error(y_train, y_hat))
print('Средняя квадратичная ошибка %.2f' % mean_squared_error(y_train, y_hat))

Можно рассмотреть более сложную меру: коэффициент детерминации $R^2$:

* $TSS = \sum_i (y^{(i)}-\bar{y})^2$ - общая сумма квадратов (total sum of squares)
* $RSS = \sum_i (\hat{y}^{(i)}-y^{(i)})^2$ - сумма квадратов остатков (residual sum of squares)
* $ESS = \sum_i (\hat{y}^{(i)}-\bar{y})^2$ - объясненная сумма квадратов (explained sum of squares)

Для простоты будем считать, что
$$TSS = ESS + RSS$$

Тогда Коэффициент детерминации $R^2=1-\frac{RSS}{TSS}$

Рассчитайте его для нашей модели


In [ ]:
r2_score(y_train, y_hat)

## Оценка значимости коэффициентов с помощью бутстрепа

#### Интро в бутстреп

Иногда для анализа данных полезно знать не только среднее значение какого-нибудь признака, но и его [доверительный интервал](http://www.machinelearning.ru/wiki/index.php?title=%D0%94%D0%BE%D0%B2%D0%B5%D1%80%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B0%D0%BB). 

Из курса статистики известно, что если выборка $x$ подчиняются нормальному закону распределения и нам известно стандартное отклонение $\sigma$ на *генеральной совокупности*, то доверительный интервал c доверительной вероятностью $(1-\alpha)$ для среднего можно вычислить по следующей формуле:

$$\left( \bar{x} - z_{1 - \alpha/2}\frac{\sigma}{\sqrt{n}}, \bar{x} + z_{1 - \alpha/2}\frac{\sigma}{\sqrt{n}} \right),$$

где $\bar{x}$ - выборочное среднее , $n$ - это размер выборки, а $z_{\gamma}$ - это квантиль нормального распределения уровня $\gamma$.

Выберем какой-то признак и посчитаем доверительный интервал

In [ ]:
x = df_train.price.values

In [ ]:
plt.hist(x, bins=20)

In [ ]:
# Посчитаем 95% доверительный интервал (alpha = 0.05)
# Тогда просто согласно формуле

sigma = x.std() # Вообще говоря, это неправда, но будем считать, что это знание свыше
n = x.shape[0]
z = 1.96 # 0.975 квантиль
xm = x.mean()

lb, rb = xm - z*sigma/np.sqrt(n), xm + z*sigma/np.sqrt(n)

print('95%% доверительный интервал: (%.3f, %.3f)' % (lb, rb))
print('Среднее %.3f' % xm)

Есть другой, более универсальный способ для расчета доверительных интервалов любых статистик - метод [bootstap](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)), идея которого заключается в многократной генерации выборок на базе имеющейся выборки.

Для того, чтобы найти доверительный интервал методом bootstrap проделайте следующие шаги:
1. Создайте случайную матрицу размера $150 \times 1000$. $150$ - потому что в iris есть измерения по $150$ объектам, а $1000$ - это количество bootstrap-выборок, которое мы отсэмплируем. Значения в этой матрице должны быть целочисленными от 0 до 149 (включительно). Полученная матрица - матрица со случайными индексами элементов массива, для которого мы считаем доверительный интервал
2. Выполните сэмплирование - должна получится новая матрица размера $150 \times 1000$, но заполненная значениями из массива.
3. Посчитайте среднее по каждому столбцу - получится массив выборочных средних.
4. По массиву из пункта выше посчитайте 2.5% и 97.5% персентили - это и будут границы доверительного интервала.

In [ ]:
# Your Code Here

Зачем это нам? Для моделей мы можем точно также делать оценки каких-то их параметров.

В частности, для линейной регрессии мы можем методом бутстрепа генерировать выборки для обучения, строить модели и запоминать полученные коэффициенты. 

Затем точно так же можно смотреть на доверительный интервал для коэффициентов

Используйте бутстреп для данного датасета

In [ ]:
# Your Code Here

Но есть различные статистические оценки для доверительных интервалов. Их можно либо считать [самому](https://math.stackexchange.com/questions/871601/confidence-interval-for-regression-coefficient-beta), либо использовать пакеты, в которых все расчитывается автоматом

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train2 = np.c_[X_train, np.ones_like(X_train)]

In [ ]:
ols = sm.OLS(y_train, exog=X_train2)

In [ ]:
model = ols.fit()

In [ ]:
model.summary2()

## Эффект регуляризации

Теперь, давайте попробуем добавить столбец с "километрами" в наш датафрейм (линейная зависимость) и 

1. Посмотрим что будет с коэффициентами
2. Добавим регуляризации

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
# Your Code Here

## Преобразование переменных

Давайте попробуем  добавить остальные переменные

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [ ]:
# Your Code Here

# Логистическая регрессия

## Игрушечный пример

Сгенерируем выборку и опробуем логистическую регрессию

In [ ]:
np.random.seed(0)
X = np.r_[np.random.randn(20, 2) + [2, 2],
          np.random.randn(20, 2) + [-2, -2]]
y = [-1] * 20 + [1] * 20

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[:, 0],
           X[:, 1],
           c=y,
           cmap=plt.cm.Paired)

In [ ]:
from sklearn.linear_model import LogisticRegression

Обучим логистическую регрессию на этих данных и нарисуем разделяющую гиперплоскость

In [ ]:
model = LogisticRegression(C=1.0, 
                           fit_intercept=True, 
                           penalty='l2')
model.fit(X, y)

In [ ]:
print('w_0 = %f' % model.intercept_)
print('w_1, w_2 = ', model.coef_)

In [ ]:
# Нарисуем эту гиперплоскость
w_0 = model.intercept_[0]
w_1 = model.coef_[0][0]
w_2 = model.coef_[0][1]

x_1 = np.linspace(-4, 4, 10)
x_2 = - (w_0 + w_1*x_1)/w_2

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[:, 0],
           X[:, 1],
           c=y,
           cmap=plt.cm.Paired)
plt.plot(x_1, x_2)

## Пример с текстами

Возьмем текстовые данные [отсюда](https://archive.ics.uci.edu/ml/machine-learning-databases/00331/). Архив содержит 3 файла с положительными и отрицательными отзывами с ресурсов
* imdb.com
* amazon.com
* yelp.com

Формат файла следующий:
<отзыв>\t<метка>\n


### Задача
1. Загрузите тексты и метки классов в разные переменные
2. Выберите меру качества классификации
3. Обучите логистическую (без подбора гиперпараметров). Тексты представляются в виде мешка слов
4. Выведите наиболее значимые слова из текста
5. С помощью кросс-валидации найдите хорошие значения гиперпараметров для `CountVectorizer` и `LogisticRegression`

In [ ]:
df = pd.read_csv('data/sentiment/imdb_labelled.txt', sep='\t', header=None, names=['text', 'label'])

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV GridSearchCV

In [ ]:
model = Pipeline([
    ('vect', CountVectorizer(min_df=4, max_df=0.95, stop_words='english', ngram_range=(1,1))),
    ('clf', LogisticRegression())
])

In [ ]:
# Your Code Here